In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow_datasets as tfds

In [14]:
def preprocess(image, label):
    image = tf.image.resize(image, (299, 299))
    image = preprocess_input(image)  # Preprocessing specific to InceptionV3
    return image, label

# Load the dataset
(train_data, test_data), dataset_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    as_supervised=True,
    with_info=True
)

# Preprocess the data
train_data = train_data.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
# Initialize the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Freeze the layers

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(dataset_info.features['label'].num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(train_data, epochs=5, validation_data=test_data)

Epoch 1/5
2368/2368 [==============================] - 143s 59ms/step - loss: 2.1027 - accuracy: 0.4696 - val_loss: 1.6164 - val_accuracy: 0.5663
Epoch 2/5
2368/2368 [==============================] - 138s 58ms/step - loss: 1.6458 - accuracy: 0.5696 - val_loss: 1.5059 - val_accuracy: 0.5909
Epoch 3/5
2368/2368 [==============================] - 138s 58ms/step - loss: 1.4761 - accuracy: 0.6089 - val_loss: 1.5061 - val_accuracy: 0.5994
Epoch 4/5
2368/2368 [==============================] - 138s 58ms/step - loss: 1.3530 - accuracy: 0.6383 - val_loss: 1.5127 - val_accuracy: 0.5985
Epoch 5/5
2368/2368 [==============================] - 137s 58ms/step - loss: 1.2504 - accuracy: 0.6612 - val_loss: 1.5703 - val_accuracy: 0.5945


In [19]:
loss, accuracy = model.evaluate(test_data)
print(f"Test accuracy: {accuracy * 100}%")

790/790 [==============================] - 34s 43ms/step - loss: 1.5703 - accuracy: 0.5945
Test accuracy: 59.445542097091675%
